In [1]:
import graphlab as gl

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.3.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [2]:
import numpy as np
import scipy as sp
import pandas as pd

# 1.Reading Data

In [213]:
features = gl.SFrame.read_csv("features.csv")

Finished parsing file /Users/sergey/Documents/Coursera/HSE/kaggle/features.csv

Parsing completed. Parsed 100 lines in 3.75693 secs.

Finished parsing file /Users/sergey/Documents/Coursera/HSE/kaggle/features.csv

Parsing completed. Parsed 97230 lines in 4.51844 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,float,float,str,float,float,float,int,int,int,int,float,float,float,float,int,int,int,int,float,int,int,int,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [214]:
features_check = gl.SFrame.read_csv("features_test.csv")

Finished parsing file /Users/sergey/Documents/Coursera/HSE/kaggle/features_test.csv

Parsing completed. Parsed 100 lines in 0.654693 secs.

Finished parsing file /Users/sergey/Documents/Coursera/HSE/kaggle/features_test.csv

Parsing completed. Parsed 17177 lines in 0.724001 secs.

------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,float,float,float,str,float,float,float,int,int,int,int,float,float,float,float,int,int,int,int,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


In [215]:
#features_check.print_rows(num_rows=3, num_columns=109)

# 2. Reshaping data

In [483]:
name_vector = features_check.column_names()[2:]
for f in name_vector:
    features = features.fillna(f,0)
data = features.to_dataframe()
y = data['radiant_win']

# 3. Enriching dataset 

In [484]:
heroes = pd.read_csv('dictionaries/heroes2.csv', ';')
type_vector = list(heroes.columns)[3:]
heroes['sum']=1.*heroes[type_vector].sum(axis=1)
for i in xrange(len(type_vector)):
    heroes[type_vector[i]]=np.divide(heroes[type_vector[i]],heroes['sum'])
r_new = []
r_old = []
d_old = []
type_vector = list(heroes.columns)[3:]
attr = ['_hero', '_level', '_xp', '_gold', '_lh', '_kills', '_deaths', '_items']
for k in type_vector:
    for i in attr:
        r_new += [k+'R'+i]
for k in type_vector:
    for i in attr:
        r_new += [k+'D'+i]
for k in range(0,5):
    for i in attr:
        r_old+=['r'+str(k+1)+i] 
for k in range(0,5):
    for i in attr:
        d_old+=['d'+str(k+1)+i] 
for i in list(r_new):
    data[i]=0
type_vector = list(heroes.columns)[3:12]
type_vector.append('match_id')
M = pd.merge(data,heroes,left_on=r_old[0],right_on='id').sort("match_id")
M.index = range(0,len(M))

In [503]:
for i in xrange(9):
    for k in xrange(8):
        x=type_vector[i]
        M[r_new[9*i+k]]=M[x]*M[r_old[k]]

In [504]:
pd.set_option('display.max_columns', 500)
print M[list(M.columns)[3:11]].head(3)
print M[type_vector].head(3)
print M[list(M.columns)[109:]].head(3)

   r1_hero  r1_level  r1_xp  r1_gold  r1_lh  r1_kills  r1_deaths  r1_items
0       11         5   2098     1489     20         0          0         7
1       42         4   1188     1033      9         0          1        12
2       33         4   1319     1270     22         0          0        12
      Carry  Initiator   Durable  Nuker  Support  Disabler    Pusher  \
0  0.500000   0.000000  0.000000    0.5        0         0  0.000000   
1  0.285714   0.000000  0.428571    0.0        0         0  0.285714   
2  0.000000   0.333333  0.000000    0.0        0         0  0.333333   

     Escape  Jungler  match_id  
0  0.000000        0         0  
1  0.000000        0         1  
2  0.333333        0         2  
   CarryR_hero  CarryR_level    CarryR_xp  CarryR_gold  CarryR_lh  \
0          5.5      2.500000  1049.000000   744.500000  10.000000   
1         12.0      1.142857   339.428571   295.142857   2.571429   
2          0.0      0.000000     0.000000     0.000000   0.000000   

  

In [ ]:
K = pd.merge(data[['match_id',d_old[0]]],heroes,left_on=d_old[0],right_on='id').sort("match_id")[type_vector]
K.index = range(0,len(K))

In [387]:
r_new = r_new[:-8]

In [388]:
N = pd.concat([M, K], axis=1)

In [389]:
for i in range(1,5):
    R = pd.merge(data[['match_id',r_old[i]]],heroes,left_on=r_old[i],right_on='id').sort("match_id")[type_vector]
    R.index = range(0,len(L))
    D = pd.merge(data[['match_id',d_old[i]]],heroes,left_on=d_old[i],right_on='id').sort("match_id")[type_vector]
    D.index = range(0,len(L))
    L = pd.concat([R, D], axis=1)
    N+=L
mult=N[list(heroes.columns)[3:-1]]

In [390]:
N.columns = ['CarryR','InitiatorR','DurableR','NukerR','SupportR','DisablerR','PusherR','EscapeR','JunglerR','del',
            'CarryD','InitiatorD','DurableD','NukerD','SupportD','DisablerD','PusherD','EscapeD','JunglerD','match_id']
del N['del']
del N['match_id']

In [396]:
name_vector = features_check.column_names()[2:]
#vector = name_vector[0:1]
#vector.extend(name_vector[82:])
data = data[name_vector]

In [397]:
data = pd.concat([data, N], axis=1)

In [400]:
data

,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_hero,...,JunglerR,CarryD,InitiatorD,DurableD,NukerD,SupportD,DisablerD,PusherD,EscapeD,JunglerD
0,7,11,5,2098,1489,20,0,0,7,67,...,0.000000,0.485714,0.485714,0.628571,0.933333,0.619048,0.485714,1.161905,0.000000,0.200000
1,0,42,4,1188,1033,9,0,1,12,49,...,0.000000,0.916667,0.428571,0.000000,1.583333,0.476190,0.142857,1.119048,0.333333,0.000000
2,7,33,4,1319,1270,22,0,0,12,98,...,0.200000,0.333333,0.333333,0.666667,1.250000,0.961111,0.000000,0.722222,0.733333,0.000000
3,1,29,4,1779,1056,14,0,0,5,30,...,0.000000,1.033333,0.333333,1.233333,0.833333,0.733333,0.166667,0.666667,0.000000,0.000000
4,7,13,4,1431,1090,8,1,0,8,27,...,0.000000,1.066667,0.000000,0.400000,1.850000,0.650000,0.200000,0.500000,0.333333,0.000000
5,1,11,5,1961,1461,19,0,1,6,20,...,0.250000,0.750000,0.458333,0.392857,1.476190,0.535714,0.553571,0.125000,0.375000,0.333333
6,1,8,3,967,1136,7,1,0,8,57,...,0.000000,0.500000,0.333333,0.500000,1.833333,1.166667,0.000000,0.666667,0.000000,0.000000
7,7,35,5,2117,1252,16,0,0,6,15,...,0.000000,1.000000,0.666667,1.000000,0.900000,0.400000,0.200000,0.833333,0.000000,0.000000
8,1,17,5,1527,906,10,0,1,7,91,...,0.250000,1.000000,0.761905,0.666667,0.666667,0.476190,0.476190,0.952381,0.000000,0.000000
9,7,15,5,1651,1060,14,0,1,10,41,...,0.000000,1.000000,0.652381,0.885714,0.375000,0.458333,0.452381,0.801190,0.375000,0.000000


# 4. Splitting train and test

In [307]:
import sklearn
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score
import sklearn.metrics

In [308]:
data['first_blood_player2']=data['first_blood_player2'].convert_objects(convert_numeric=True)
data['first_blood_player2']=data['first_blood_player2'].fillna(0)
Xf = data
Xf_train, Xf_test, y_train, y_test = train_test_split(Xf, y, test_size=0.1, random_state=42)

# 5. Logistics regression

In [309]:
from sklearn.linear_model import LogisticRegression

In [310]:
model1f = LogisticRegression(penalty='l1', tol=0.01, random_state=42)

In [311]:
model1f.fit(Xf_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l1', random_state=42, solver='liblinear', tol=0.01,
          verbose=0)

In [312]:
model1f.score(Xf_test,y_test)

0.5990949295484933

In [276]:
#model1f.coef_

# 7. Gradient boosting